## Word Vectors

Ref: https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors

Goal: Using distributed word vectors created by the Word2Vec algorithm. Word2Vec (Google, 2013) is a neural network implementation that learns distributed representations for words. It does not need labels.
Using word2vec in Python

We will use gensim package. We also need to install cython otherwise it will take days to run instead of minutes.
## Preparing to Train a Model

In [40]:
import pandas as pd
import time
start = time.time()
# Read data from files
train = pd.read_csv('labeledTrainData.tsv',header=0,delimiter='\t',quoting=3)
test = pd.read_csv('testData.tsv',header=0,delimiter='\t',quoting=3)
unlabeled_train=pd.read_csv('unlabeledTrainData.tsv',header=0,delimiter='\t',quoting=3)
# verify the number of the review that were read (100,000 in total)
print ("Read %d labeled traini reviews, %d labeled test reviews," \
"and %d unlabeled reviews\n" %(train['review'].size, test['review'].size, unlabeled_train['review'].size))

Read 25000 labeled traini reviews, 25000 labeled test reviews,and 50000 unlabeled reviews



In [41]:
# Import various modules for string cleaning 
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review,remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words. Returns a list of words.
    # 1. Remove HTML
    review_text=BeautifulSoup(review,"html").get_text()
    # 2. Remove non-letters, keep numbers
    review_text=re.sub("[^a-zA-Z0-9]"," ",review_text)
    # 3. Convert words in lower case and split them
    words=review_text.lower().split()
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops=set(stopwords.words("english"))
        words=[w for w in words if not w in stops]
    # 5. return a list of words
    return(words)

In [42]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download()

In [43]:
# load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [44]:
# Define a function to split a review into parsed sentences
def review_to_sentences(review,tokenizer,remove_stopwords=False):
    # Function to split a review into parsed sentences. 
    # Returns a list of sentences, where each sentence is a list of words.
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences=tokenizer.tokenize(review.strip())
    # 2. Loop over each sentence
    sentences=[]
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence)>0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence,remove_stopwords))
        # Return the list of sentences (each sentence is a list of words, so this returns a list of lists)
    return sentences

In [45]:
sentences = []

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences+=review_to_sentences(review, tokenizer)
    
print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences+=review_to_sentences(review, tokenizer)


Parsing sentences from training set


/home/amywu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/amywu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:198: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/amywu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.happierabroad.com"'" looks like a URL. Beautiful Soup 

Parsing sentences from unlabeled set


/home/amywu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.archive.org/details/LovefromaStranger"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/amywu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.loosechangeguide.com/LooseChangeGuide.html"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup

In [46]:
# Check how many sentences we have in total- should be around 850,000+
print (len(sentences))
print (sentences[0])
print (sentences[1])



795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


## Training and Saving Your Model
- **Architecture**: Skip-gram (default) or continuous bags of words. Skip-gram slightly slower but produced better results
- **Training algorithm**: Hierarchical softmax (default) or negative sampling. Default works better
- **Downsamping of frequent words**: Google documentation recommends between .00001 and .001. Values closer .001 seemed to improve the accuracy of the final model
- **Word vector dimensionality**: Reasonable values can be in the tens to hundreds. We used 300.
- **Context/window size**: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax
- **Worker threads**: Number of parallel processes to run
- **Minimum word count**: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words.

In [47]:
# Import the built-in logging module and configure it so that Word2Vec create nice output message
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s: %(message)s',level=logging.INFO)
# Set values for various parameters:
num_features =300 # word vector dimensionality
min_word_count =40 # minimum word count
num_workers =4 # Number of threads to run in parallel
context = 10 # Context window size
downsampling = 1e-3 # Downsample setting for frequent words
# Initialize and train the model
from gensim.models import word2vec
print('Training Model...')
model = word2vec.Word2Vec(sentences,workers=num_workers,size=num_features,min_count=min_word_count,window=context,\
                          sample=downsampling)
#If you don't plan to train the model any further, calling init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)
#It can be helpful to create a meaningful model name and save the model for latter use. You can load it later using Word2Vec.load()
model_name='300features_40minwords_10context'
model.save(model_name)
end = time.time()
print('{} sec'.format((end-start)))

Training Model...
342.69152307510376 sec


## Exploring the Model Results
The 'doesnt_match' function will try to deduce which word in a set is most dissimilar from the others:

In [48]:
model.doesnt_match('man woman child kitchen'.split())

'kitchen'

In [49]:
model.doesnt_match('france england germany berlin'.split())

'berlin'

In [50]:
model.doesnt_match('paris berlin london austria'.split())

'paris'

In [51]:
# Function "most_similar" can get insight into the model's word clusters:
model.most_similar("man")

[('woman', 0.6193481683731079),
 ('lady', 0.5889842510223389),
 ('lad', 0.5411475896835327),
 ('men', 0.5185449123382568),
 ('millionaire', 0.5149509310722351),
 ('guy', 0.5141556262969971),
 ('monk', 0.5101891756057739),
 ('farmer', 0.5047257542610168),
 ('businessman', 0.5012615919113159),
 ('sailor', 0.5010964274406433)]

In [52]:
model.most_similar("queen")

[('princess', 0.6624864339828491),
 ('bride', 0.6191389560699463),
 ('mistress', 0.6012598276138306),
 ('victoria', 0.5922946929931641),
 ('kristel', 0.5868010520935059),
 ('showgirl', 0.5861167311668396),
 ('stepmother', 0.582958459854126),
 ('latifah', 0.5828735828399658),
 ('goddess', 0.5825923681259155),
 ('belle', 0.5806471705436707)]

In [53]:
model.most_similar('awful')

[('terrible', 0.7759661674499512),
 ('atrocious', 0.758520245552063),
 ('horrible', 0.7496703863143921),
 ('abysmal', 0.7101701498031616),
 ('horrendous', 0.7023831605911255),
 ('dreadful', 0.6928654313087463),
 ('appalling', 0.6835609078407288),
 ('horrid', 0.6614154577255249),
 ('lousy', 0.6244205236434937),
 ('amateurish', 0.615602970123291)]